In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/tfnet/')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/tfloss/')
sys.path.append('/home/macul/libraries/mk_utils/tf_spoof/')
sys.path.append('/home/macul/libraries/mk_utils/spoofing_lbp/')
from config import net_config as config
import argparse
import logging
import json
import os
from os.path import isdir
from os import mkdir
from shutil import copyfile
from spoof_feature import LBP_GREY, LBP_RG, CoALBP, CoALBP_256, CoALBP_GREY, BSIF, LPQ, COLOR_MOMENT
from tfspoofdense import SpoofDenseNet
from tfloss import TfLosses
import time
from tensorflow.core.protobuf import config_pb2
from PIL import Image
import pickle

#tf.enable_eager_execution()
Model_Process = {'BSIF'        : BSIF(),
                 'CoALBP'      : CoALBP(),
                 'CoALBP_GREY' : CoALBP_GREY(),
                 'CoALBP_256'  : CoALBP_256(),
                 'COLOR_MOMENT': COLOR_MOMENT(),
                 'LBP_GREY'    : LBP_GREY(),
                 'LBP_RG'      : LBP_RG(),
                 'LPQ'         : LPQ(),
                }

# get ft extractor and network configuration
ft_extractor = [Model_Process[ex] for ex in config.Feature_Type]
ft_layer_units = [config.Dense_Cfg[ex] for ex in config.Feature_Type]

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.DEVICE_IDS
#batchSize = config.BATCH_SIZE * config.NUM_DEVICES

In [3]:
def img_crop_shuffle(img, height, width, grid_size=4):
    crop_h = tf.cast(height/4,tf.int32)
    crop_w = tf.cast(width/4,tf.int32)
    
    idx = list(range(grid_size*grid_size))
    np.random.shuffle(idx)
    img_tmp = []
    cnt = 0
    for i, v in enumerate(idx):
        if i%grid_size == 0:
            img_crop = tf.image.crop_to_bounding_box(img, int(idx[i]/4)*crop_h,int(idx[i]%4)*crop_w,crop_h,crop_w)
        else:
            img_crop = tf.concat([img_crop, tf.image.crop_to_bounding_box(img, int(idx[i]/4)*crop_h,int(idx[i]%4)*crop_w,crop_h,crop_w)], 1)
        cnt += 1
        if cnt == grid_size:
            img_tmp += [img_crop]
            cnt = 0
    img1 = img_tmp[0]
    for i in range(1,len(img_tmp)):
        img1 = tf.concat([img1,img_tmp[i]],0)
        
    return img1


# this function must be used for batch_size of 1 or before batch operation since the image size varies
def parse_function(example_proto, grid_size=4):
    
    features = {'image/encoded': tf.FixedLenFeature([], tf.string),
                'image/height': tf.FixedLenFeature([], tf.int64),
                'image/width': tf.FixedLenFeature([], tf.int64),
                #'image/colorspace': tf.FixedLenFeature([], tf.string),
                #'image/channels': tf.FixedLenFeature([], tf.int64),
                #'image/class/text': tf.FixedLenFeature([], tf.string),
                'image/class/label': tf.FixedLenFeature([], tf.int64)}
    
    features = tf.parse_single_example(example_proto, features)
    
    img = tf.image.decode_jpeg(features['image/encoded'])
    #img = features['image/encoded']
    
    # img = tf.reshape(img, shape=(112, 112, 3))
    # r, g, b = tf.split(img, num_or_size_splits=3, axis=-1)
    # img = tf.concat([b, g, r], axis=-1)
    
    #img = tf.cast(img, dtype=tf.float32)
    #img = tf.subtract(img, 127.5)
    #img = tf.multiply(img,  0.0078125)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    
    label = tf.cast(features['image/class/label'], tf.int64) - 1
    
    if grid_size <= 1:
        return img, label
    
    height = tf.cast(features['image/height'], tf.int64)
    width = tf.cast(features['image/width'], tf.int64)
    
    return img_crop_shuffle(img,height,width,grid_size), label

def feature_extraction(img, extractor):
    ft = []
    for ex in extractor:
        ft += [tf.cast(tf.py_func(ex, [img], tf.double),tf.float32)]
    return tuple(ft) # list does not work here, need to convert to tuple
    #return tuple(ft), img # for dataset debug

In [4]:
'''
# dataset testing
dataset_train = tf.data.TFRecordDataset(config.TRAIN_REC)

#for raw_record in dataset_train.take(1):
#    print(repr(raw_record))

dataset_train = dataset_train.map(lambda x: parse_function(x, config.Img_shuffle_grid_size))
#dataset_train = dataset_train.shuffle(buffer_size=config.BUFFER_SIZE) # shuffle the whole dataset is better
#dataset_train = dataset_train.batch(1)
#dataset_train = dataset_train.apply(tf.data.experimental.unbatch())
dataset_train = dataset_train.map(lambda *x: (feature_extraction(x[0], ft_extractor), x[1]))
dataset_train = dataset_train.batch(4)
iterator_train = dataset_train.make_initializable_iterator()
next_element_train = iterator_train.get_next()

cfg = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
cfg.gpu_options.allow_growth = True

sess = tf.Session(config=cfg)
sess.run(iterator_train.initializer)

feature, label = sess.run(next_element_train)
'''

#training_ph = tf.placeholder(name='training_flag',shape=[], dtype=tf.bool)

'\n# dataset testing\ndataset_train = tf.data.TFRecordDataset(config.TRAIN_REC)\n\n#for raw_record in dataset_train.take(1):\n#    print(repr(raw_record))\n\ndataset_train = dataset_train.map(lambda x: parse_function(x, config.Img_shuffle_grid_size))\n#dataset_train = dataset_train.shuffle(buffer_size=config.BUFFER_SIZE) # shuffle the whole dataset is better\n#dataset_train = dataset_train.batch(1)\n#dataset_train = dataset_train.apply(tf.data.experimental.unbatch())\ndataset_train = dataset_train.map(lambda *x: (feature_extraction(x[0], ft_extractor), x[1]))\ndataset_train = dataset_train.batch(4)\niterator_train = dataset_train.make_initializable_iterator()\nnext_element_train = iterator_train.get_next()\n\ncfg = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)\ncfg.gpu_options.allow_growth = True\n\nsess = tf.Session(config=cfg)\nsess.run(iterator_train.initializer)\n\nfeature, label = sess.run(next_element_train)\n'

In [5]:
net_scope = 'SpoofDenseNet'

# declare placeholders
labels_ph = tf.placeholder(name='label',shape=[None,], dtype=tf.int64)
features_ph = tuple([tf.placeholder(name='ft_{}'.format(f_key),shape=[None,config.Feature_Size[f_key]], 
                                    dtype=tf.float32) for f_key in config.Feature_Type])
# build training dataset and network
dataset_train = tf.data.TFRecordDataset(config.TRAIN_REC)
dataset_train = dataset_train.map(lambda x: parse_function(x, config.Img_shuffle_grid_size))
dataset_train = dataset_train.shuffle(buffer_size=config.BUFFER_SIZE) # shuffle the whole dataset is better
dataset_train = dataset_train.map(lambda *x: (feature_extraction(x[0], ft_extractor), x[1]))
# remember to set batch size to 1 for dataset debug
#dataset_train = dataset_train.map(lambda *x: feature_extraction(x[0], ft_extractor) + (x[1],)) # for dataset debug
dataset_train = dataset_train.batch(config.BATCH_SIZE)
iterator_train = dataset_train.make_initializable_iterator()
next_element_train = iterator_train.get_next()
with tf.variable_scope(net_scope):
    # build base network
    embeddings = SpoofDenseNet.build(features_ph, feature_units=ft_layer_units, stem_units=config.Dense_Stem_Cfg, 
                                     training=True, act=config.Activation, reg=config.Regularizer, 
                                     init=config.Initializer, scope='spoof')
    #logit, inference_loss = TfLosses.softmax_loss(embedding=embeddings, labels=labels_ph, out_num=config.NUM_CLASSES,  
    #                                              act=config.Activation, reg=config.Regularizer, 
    #                                              init=config.Initializer)
    logit, inference_loss = TfLosses.arc_loss(embedding=embeddings, labels=labels_ph, w_init=config.Initializer, 
                                              out_num=config.NUM_CLASSES, s=config.Arc_margin_scale, 
                                              m=config.Arc_margin_angle)
    pred = tf.nn.softmax(logit)
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, axis=1), labels_ph), dtype=tf.float32))

In [6]:
# prepare for training
global_step = tf.Variable(name='global_step', initial_value=0, trainable=False)
inc_op = tf.assign_add(global_step, 1, name='increment_global_step')
lr = tf.train.piecewise_constant(global_step, boundaries=config.Opt_lr_steps, values=config.Opt_lr, name='lr_schedule')
# define the optimize method
opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=config.Opt_momentum)
# get train op
#grads = opt.compute_gradients(inference_loss)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):        
    #train_op = opt.apply_gradients(grads, global_step=global_step)
    train_op = opt.minimize(inference_loss, global_step=global_step)
    

In [7]:
# declare placeholders
labels_val_ph = tf.placeholder(name='label',shape=[None,], dtype=tf.int64)
features_val_ph = tuple([tf.placeholder(name='ft_{}'.format(f_key),shape=[None,config.Feature_Size[f_key]], 
                                    dtype=tf.float32) for f_key in config.Feature_Type])
# build validation dataset and network
dataset_val = tf.data.TFRecordDataset(config.VAL_REC)
dataset_val = dataset_val.map(lambda x: parse_function(x, 0))
dataset_val = dataset_val.map(lambda *x: (feature_extraction(x[0], ft_extractor), x[1]))
dataset_val = dataset_val.batch(config.BATCH_SIZE)
iterator_val = dataset_val.make_initializable_iterator()
next_element_val = iterator_val.get_next()
with tf.variable_scope(net_scope, reuse=True):
    # build base network
    embeddings_val = SpoofDenseNet.build(features_val_ph, feature_units=ft_layer_units, stem_units=config.Dense_Stem_Cfg, 
                                         training=False, act=config.Activation, reg=config.Regularizer, 
                                         init=config.Initializer, scope='spoof')
    logit_val, _ = TfLosses.arc_loss(embedding=embeddings_val, labels=labels_val_ph, w_init=config.Initializer, 
                                     out_num=config.NUM_CLASSES, s=config.Arc_margin_scale, m=0)
    pred_val = tf.nn.softmax(logit_val)
    acc_val = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred_val, axis=1), labels_val_ph), dtype=tf.float32))

In [8]:
# create session
cfg = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
cfg.gpu_options.allow_growth = True
sess = tf.Session(config=cfg)
# create model saver
saver = tf.train.Saver()

time_stamp = time.strftime('_%Y-%m-%d-%H-%M', time.localtime(time.time()))

# create output folder
checkpointsPath = os.path.sep.join([config.OUT_PATH, config.PREFIX])
if not isdir(checkpointsPath):
    mkdir(checkpointsPath)
    
# define log file
log_file_path = checkpointsPath + '/train' + time_stamp + '.log'
log_file = open(log_file_path, 'w')

# summary writer
summary = tf.summary.FileWriter(checkpointsPath, sess.graph)
summaries = []
# trainabel variable gradients
for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))
# add loss summary
summaries.append(tf.summary.scalar('inference_loss', inference_loss))
# add learning rate
summaries.append(tf.summary.scalar('leraning_rate', lr))
# add accuracy
summaries.append(tf.summary.scalar('accuracy', acc))
summary_op = tf.summary.merge(summaries)

In [9]:
img_verify_flag = False # set to false during actual training
count = 0
validation_result = []

sess.run(tf.global_variables_initializer())
for i in range(config.NUM_EPOCH):
    
    sess.run(iterator_train.initializer)
    while True:
        try:
            start = time.time()
            images_train, labels_train = sess.run(next_element_train)
            #images_train, images_show, labels_train = sess.run(next_element_train) # for dataset debug
            if img_verify_flag: # for dataset debug
                img = Image.fromarray(images_show[0,...], 'RGB')
                img.show()
                exit(1)
            feed_dict = {features_ph: images_train, labels_ph: labels_train}            
            embTrain, logitTrain, inferenceLossTrain, accTrain, _, _ = \
                sess.run([embeddings, logit, inference_loss, acc, train_op, inc_op],
                          feed_dict=feed_dict,
                          options=config_pb2.RunOptions(report_tensor_allocations_upon_oom=True))
            end = time.time()
            pre_sec = config.BATCH_SIZE/(end - start)
            
            if count > 0 and count % config.Update_Interval == 0:
                # logging
                print('Training: epoch %d, total_step %d, inference loss is %.2f, '
                      'accuracy is %.6f, time %.3f samples/sec' %
                          (i, count, inferenceLossTrain, accTrain,pre_sec))
                log_file.write('Training: epoch %d, total_step %d, inference_loss %.2f, '
                               'accuracy %.6f, time %.3f samples/sec' %
                               (i, count, inferenceLossTrain, accTrain,pre_sec) + '\n')
                log_file.flush()
                
                # save summary
                summary_op_val = sess.run(summary_op, feed_dict=feed_dict)
                summary.add_summary(summary_op_val, count)
                
                #print(embTrain)
                #print(logitTrain)
                #print(inferenceLossTrain)
                #print(labels_train)
                #print(images_train)
            count += 1
        except tf.errors.OutOfRangeError:
            print("End of epoch %d" % i)
            break
    
    # save check points
    ckpt_filename = config.PREFIX+'_{:d}'.format(i) + '.ckpt'
    ckpt_filename = os.path.join(checkpointsPath, ckpt_filename)
    saver.save(sess, ckpt_filename)
    
            
    # do validation
    accVal = []
    predVal = []
    labelVal = np.array([])
    sess.run(iterator_val.initializer)
    while True:
        try:
            images_val, labels_val = sess.run(next_element_val)
            feed_dict = {features_val_ph: images_val, labels_val_ph: labels_val}
            acc_tmp, pred_tmp = \
                sess.run([acc_val, pred_val],
                          feed_dict=feed_dict,
                          options=config_pb2.RunOptions(report_tensor_allocations_upon_oom=True))                        
            accVal += [acc_tmp]
            #print(accVal)
            
            if type(predVal) == type([]):
                predVal = pred_tmp
            else:
                predVal = np.append(predVal, pred_tmp, axis=0)
            labelVal = np.append(labelVal, labels_val)
            
            #print(predVal)
            #print(labelVal)
        except tf.errors.OutOfRangeError:
            break    
    accVal = np.mean(accVal)
    print('$$$$$$$$ Validation: epoch %d, accuracy is %.6f' % (i, accVal))
    log_file.write('Validation: epoch %d, accuracy %.6f' % (i, accVal) + '\n')
    log_file.flush()
    
    # save validation results
    validation_result += [{'label': labelVal, 'pred': predVal}]
    with open(checkpointsPath+'/'+config.PREFIX+'_val_result.pkl', 'wb') as f:
        pickle.dump(validation_result, f)
    

Training: epoch 0, total_step 10, inference loss is 6.63, accuracy is 0.390625, time 313.202 samples/sec
Training: epoch 0, total_step 20, inference loss is 4.69, accuracy is 0.453125, time 295.324 samples/sec
Training: epoch 0, total_step 30, inference loss is 3.50, accuracy is 0.468750, time 310.603 samples/sec
Training: epoch 0, total_step 40, inference loss is 2.66, accuracy is 0.429688, time 310.280 samples/sec
Training: epoch 0, total_step 50, inference loss is 1.61, accuracy is 0.484375, time 315.851 samples/sec
Training: epoch 0, total_step 60, inference loss is 1.42, accuracy is 0.437500, time 312.414 samples/sec
Training: epoch 0, total_step 70, inference loss is 1.05, accuracy is 0.507812, time 307.085 samples/sec
Training: epoch 0, total_step 80, inference loss is 0.98, accuracy is 0.539062, time 311.877 samples/sec
Training: epoch 0, total_step 90, inference loss is 0.95, accuracy is 0.539062, time 296.226 samples/sec
Training: epoch 0, total_step 100, inference loss is 0.

End of epoch 1
$$$$$$$$ Validation: epoch 1, accuracy is 0.708231
Training: epoch 2, total_step 780, inference loss is 0.88, accuracy is 0.625000, time 281.982 samples/sec
Training: epoch 2, total_step 790, inference loss is 0.89, accuracy is 0.648438, time 290.850 samples/sec
Training: epoch 2, total_step 800, inference loss is 0.86, accuracy is 0.625000, time 297.966 samples/sec
Training: epoch 2, total_step 810, inference loss is 0.81, accuracy is 0.671875, time 290.259 samples/sec
Training: epoch 2, total_step 820, inference loss is 0.72, accuracy is 0.765625, time 280.049 samples/sec
Training: epoch 2, total_step 830, inference loss is 0.90, accuracy is 0.578125, time 294.991 samples/sec
Training: epoch 2, total_step 840, inference loss is 0.92, accuracy is 0.601562, time 296.708 samples/sec
Training: epoch 2, total_step 850, inference loss is 0.88, accuracy is 0.585938, time 299.805 samples/sec
Training: epoch 2, total_step 860, inference loss is 0.97, accuracy is 0.531250, time 

Training: epoch 3, total_step 1540, inference loss is 0.91, accuracy is 0.562500, time 293.708 samples/sec
Training: epoch 3, total_step 1550, inference loss is 0.84, accuracy is 0.679688, time 295.243 samples/sec
End of epoch 3
$$$$$$$$ Validation: epoch 3, accuracy is 0.657903
Training: epoch 4, total_step 1560, inference loss is 0.86, accuracy is 0.632812, time 308.375 samples/sec
Training: epoch 4, total_step 1570, inference loss is 0.87, accuracy is 0.617188, time 302.404 samples/sec
Training: epoch 4, total_step 1580, inference loss is 0.93, accuracy is 0.593750, time 307.009 samples/sec
Training: epoch 4, total_step 1590, inference loss is 0.98, accuracy is 0.539062, time 291.729 samples/sec
Training: epoch 4, total_step 1600, inference loss is 1.00, accuracy is 0.539062, time 302.809 samples/sec
Training: epoch 4, total_step 1610, inference loss is 0.81, accuracy is 0.687500, time 301.428 samples/sec
Training: epoch 4, total_step 1620, inference loss is 0.82, accuracy is 0.6328

Training: epoch 5, total_step 2300, inference loss is 0.96, accuracy is 0.578125, time 299.199 samples/sec
Training: epoch 5, total_step 2310, inference loss is 0.84, accuracy is 0.648438, time 286.085 samples/sec
Training: epoch 5, total_step 2320, inference loss is 0.81, accuracy is 0.632812, time 291.711 samples/sec
Training: epoch 5, total_step 2330, inference loss is 0.79, accuracy is 0.671875, time 278.380 samples/sec
End of epoch 5
$$$$$$$$ Validation: epoch 5, accuracy is 0.733665
Training: epoch 6, total_step 2340, inference loss is 0.86, accuracy is 0.585938, time 312.012 samples/sec
Training: epoch 6, total_step 2350, inference loss is 0.96, accuracy is 0.578125, time 297.012 samples/sec
Training: epoch 6, total_step 2360, inference loss is 0.83, accuracy is 0.679688, time 313.450 samples/sec
Training: epoch 6, total_step 2370, inference loss is 0.83, accuracy is 0.609375, time 305.913 samples/sec
Training: epoch 6, total_step 2380, inference loss is 0.78, accuracy is 0.6640

Training: epoch 7, total_step 3060, inference loss is 0.68, accuracy is 0.742188, time 297.474 samples/sec
Training: epoch 7, total_step 3070, inference loss is 0.86, accuracy is 0.593750, time 303.640 samples/sec
Training: epoch 7, total_step 3080, inference loss is 0.86, accuracy is 0.632812, time 291.187 samples/sec
Training: epoch 7, total_step 3090, inference loss is 0.90, accuracy is 0.554688, time 292.975 samples/sec
Training: epoch 7, total_step 3100, inference loss is 0.91, accuracy is 0.578125, time 297.230 samples/sec
Training: epoch 7, total_step 3110, inference loss is 0.88, accuracy is 0.625000, time 299.767 samples/sec
End of epoch 7
$$$$$$$$ Validation: epoch 7, accuracy is 0.722534
Training: epoch 8, total_step 3120, inference loss is 0.85, accuracy is 0.648438, time 295.369 samples/sec
Training: epoch 8, total_step 3130, inference loss is 0.77, accuracy is 0.664062, time 302.460 samples/sec
Training: epoch 8, total_step 3140, inference loss is 0.75, accuracy is 0.6875

Training: epoch 9, total_step 3820, inference loss is 0.90, accuracy is 0.601562, time 240.750 samples/sec
Training: epoch 9, total_step 3830, inference loss is 0.86, accuracy is 0.664062, time 241.750 samples/sec
Training: epoch 9, total_step 3840, inference loss is 0.91, accuracy is 0.546875, time 233.540 samples/sec
Training: epoch 9, total_step 3850, inference loss is 0.82, accuracy is 0.632812, time 240.865 samples/sec
Training: epoch 9, total_step 3860, inference loss is 0.90, accuracy is 0.570312, time 234.743 samples/sec
Training: epoch 9, total_step 3870, inference loss is 0.83, accuracy is 0.695312, time 298.041 samples/sec
Training: epoch 9, total_step 3880, inference loss is 0.93, accuracy is 0.632812, time 294.203 samples/sec
End of epoch 9
$$$$$$$$ Validation: epoch 9, accuracy is 0.729134


In [ ]:
feed_dict = {features_ph: feature}

In [ ]:
sess.run(tf.global_variables_initializer())
emb_out=sess.run([embedding],feed_dict=feed_dict)

In [ ]:
emb_out[0].shape

In [ ]:
aa = tf.cast(0.1, dtype=tf.bool)
aaa=sess.run(aa)
print(aaa)

In [ ]:
print(len(net),net[0].shape,net[1].shape)

In [ ]:
net1 = SpoofDenseNet.build(net, [128,64,16], True, tf.nn.relu, config.Regularizer['CoALBP_GREY'], config.Initializer, scope="firstTry1")

In [ ]:
print(images[1].shape,label)

In [ ]:
print(labels, height, width)

In [ ]:
from PIL import Image
img = Image.fromarray(images[0,...], 'RGB')
img.show()

In [ ]:
print(features['image/height'][0])
print(features['image/width'][0])
print(features['image/class/text'][0])
print(features['image/class/label'][0])
print(len(features['image/encoded'][0]))

In [ ]:
type(images)

In [ ]:
images[0].shape

In [ ]:
img = tf.image.decode_jpeg(features['image/encoded'][0])

In [ ]:
a=sess.run(img)

In [ ]:
a.shape

In [ ]:
import tensorflow as tf

In [ ]:
global_step = tf.Variable(name='global_step', initial_value=0, trainable=False)
inc_op = tf.assign_add(global_step, 1, name='increment_global_step')
lr = tf.train.piecewise_constant(global_step, boundaries=config.Opt_lr_steps, values=config.Opt_lr, name='lr_schedule')
# define the optimize method
opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=config.Opt_momentum)
# get train op
#grads = opt.compute_gradients(inference_loss)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):        
    #train_op = opt.apply_gradients(grads, global_step=global_step)
    train_op = opt.minimize(inference_loss, global_step=global_step)

In [ ]:

tf.reset_default_graph()

is_training = tf.placeholder_with_default(False, (), 'is_training')
x = tf.placeholder(tf.float32, [None, 1], 'x')
y = tf.layers.batch_normalization(x, training=True)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    y = tf.identity(y)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

x_1 = [[-10], [0], [10]]
x_2 = [[-10]]
for _ in range(1000):
    y_1 = sess.run(y, feed_dict={x: x_1, is_training: False})
y_2 = sess.run(y, feed_dict={x: x_2})

In [ ]:
y_1

In [ ]:
y_2